In [166]:
using Symbolics
using LinearAlgebra

In [54]:
function Flux(flu,par)
    χ = par
    #flux = zeros(5)
    μ = -flu[1]  
    T = (abs(μ))^(-1//2) 
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = -2χ₁ * x3 * T / (γ*μ^3) - 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) - 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = -10χ₀ * x2 * T / μ^3 - 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    #flux[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    #flux[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    #flux[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #flux[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #flux[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    #return -flux[:]
    return [4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v; 
            4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ;
            -χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4;
            -χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4;
            -3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4]
end

Flux (generic function with 2 methods)

In [55]:
@variables f[1:5], c[1:5], p[1:3]#, fl[1:5]

JFS = Symbolics.jacobian(Flux(f,p),f);
JF_exp = Symbolics.build_function(JFS, f, p);
JFac = eval(JF_exp[1]);

In [95]:
@variables x1, x2, x3

flu=[-1.;0;0;0;0];
con=ones(5)
χ=[-1.,-0.5,-5.]

3-element Vector{Float64}:
 -1.0
 -0.5
 -5.0

In [57]:
JFac(flu,p)

5×5 Matrix{Num}:
  0.0    (-8//1)*p₁          0.0    -10.0p₁      0.0
 -4.0p₁           0.0       -2.0p₂    0.0       -2.0p₂
  0.0            -5.0p₂      0.0    -60.0p₃      0.0
 -2.5p₂           0.0    -12p₃        0.0    -12p₃
  0.0            -3.0p₂      0.0    -36.0p₃      0.0

In [58]:
include("inversion_ext.jl")

split_u!

In [96]:
flu[1] = -flu[1]
con = F(flu,con, p)
#flu[1] = -flu[1];

In [105]:
Jac(flu,con,p)

5×5 Matrix{Num}:
         12.0p₁           0.0       -6.0p₂      0.0        0.0
          0.0    (-8//1)*p₁          0.0    -10p₁          0.0
 (15//2)*p₂              -0.0      -36.0p₃     -0.0        0.0
          0.0           -15.0p₂     -0.0      -60.0p₃     -0.0
 (15//2)*p₂               0.0    -12p₃         -0.0    -12p₃

In [106]:
P(flu,c,p) = Jac(flu,c,p) \ JFac(flu,p)

P (generic function with 1 method)

In [164]:
P(flu,c,p)

5×5 Matrix{Num}:
                  0.0                                              …                                              0.0
 (-1//8)*(p₁^-1)*(4.0p₁ - (100.0p₁*p₂*((18.75p₂ - (60.0p₃))^-1)))     (-1//8)*(p₁^-1)*((2//1)*p₂ + 10p₁*(-12p₃ - (3.75(p₁^-1)*(p₂^2)))*((18.75p₂ - (60.0p₃))^-1))
                  0.0                                                                                            -0.0
                -10.0p₂*((18.75p₂ - (60.0p₃))^-1)                                                       (-12p₃ - (3.75(p₁^-1)*(p₂^2)))*((18.75p₂ - (60.0p₃))^-1)
                  0.0                                                                                            -0.0

In [107]:
P(flu,c,[-1,0,-1])

5×5 Matrix{Float64}:
 -0.0  -0.666667  -0.0    0.833333  -0.0
 -0.5   0.0       -0.25  -0.0       -0.25
  0.0   0.0        0.0    1.66667    0.0
  0.0   0.0        0.2    0.0        0.2
  0.0   0.0        0.0    1.33333    0.0

In [108]:
M = [0 -2//3 0 5//6 0;
    -1//2 0 -1//4 0 -1//4;
    0 0 0 5//3 0;
    0 0 1//5 0 1//5;
    0 0 0 4//3 0]

5×5 Matrix{Rational{Int64}}:
  0//1  -2//3   0//1  5//6   0//1
 -1//2   0//1  -1//4  0//1  -1//4
  0//1   0//1   0//1  5//3   0//1
  0//1   0//1   1//5  0//1   1//5
  0//1   0//1   0//1  4//3   0//1

In [109]:
P(flu,c,[-1,0,-1]) - M

5×5 Matrix{Float64}:
 -0.0  0.0  -0.0  -1.11022e-16  -0.0
  0.0  0.0   0.0  -0.0           0.0
  0.0  0.0   0.0  -2.22045e-16   0.0
  0.0  0.0   0.0   0.0           0.0
  0.0  0.0   0.0   0.0           0.0

In [110]:

eigvals(M)

5-element Vector{Float64}:
 -0.7745966692414842
 -0.5773502691896256
  3.0357660829594124e-18
  0.5773502691896258
  0.7745966692414837

In [177]:
eigvals(P(flu,c,[-1,-0.5,-1]))

5-element Vector{Float64}:
 -1.1370084548097363
 -0.4303061738771479
  1.7551571048406045e-18
  0.4303061738771482
  1.1370084548097377

In [178]:
function Is(flu,par)
    χ = par #only used to test otherwise are defined below. a is and amplitude to change I from outside
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = χ₀*μ^5/(15π*χ₁^2) # OK 
    λ = -χ₀*μ^4/(1π*χ₁^2)   # OK
    T = (μ)^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1. - v^2)^(-1//2)
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            -2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            -2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            -2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            ]
end

Is (generic function with 1 method)

In [179]:
Is(f,p)

5-element Vector{Num}:
 0.0
 0.0
 3.141592653589793f₅*(f₁^(-9//2))*(f₂^2)*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(1//2)) - (94.24777960769379f₂*f₄*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(-1//2))) - (3.141592653589793f₃*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(1//2))*((1//10) - ((2//5)*((1.0 - (f₂^2))^(-1//1)))))
 1.2566370614359172f₂*f₃*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(-1//2)) + 3.141592653589793f₂*f₅*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(1//2)) - (47.12388980384689f₄*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*(1 + f₂^2)*((1.0 - (f₂^2))^(-1//2)))
 3.141592653589793f₅*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(1//2)) - (94.24777960769379f₂*f₄*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(-1//2))) - (3.141592653589793f₃*(f₁^(-9//2))*(p₁^-1)*(p₂^2)*((1.0 - (f₂^2))^(1//2))*((-1//10) - ((2//5)*(f₂^2)*((1.0 - (f₂^2))^(-1//1)))))

In [180]:
JSS = Symbolics.jacobian(Is(f,p),f);
JS_exp = Symbolics.build_function(JSS, f, p);
JSac = eval(JS_exp[1]);

In [181]:
flu=[1.;0;0;0;0];
con=ones(5)
JSac(flu,p)

5×5 Matrix{Num}:
 0    0    0                       …  0
 0    0    0                          0
 0.0  0.0  0.942478(p₁^-1)*(p₂^2)     0.0
 0.0  0.0  0.0                        0.0
 0.0  0.0  0.314159(p₁^-1)*(p₂^2)     3.14159(p₁^-1)*(p₂^2)

In [182]:
II(flu,c,p) = Jac(flu,c,p) \ JSac(flu,p)

II (generic function with 1 method)

In [183]:
II(flu,c,p)

5×5 Matrix{Num}:
 0    0    …   0
 0    0        0
 0.0  0.0      0.0
 0.0  0.0      0.0
 0.0  0.0     -0.261799(p₁^-1)*(p₂^2)*(p₃^-1)

In [202]:
par = [-1,-0.50,-5]
eigvals(0.0*im .* P(flu,c,par) - II(flu,c,par))

5-element Vector{ComplexF64}:
  -0.04053667940115862 + 0.0im
                   0.0 - 0.0im
                   0.0 + 0.0im
 0.0013158503261109084 + 0.0im
   0.01308996938995747 + 0.0im